In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.datasets import imdb

In [3]:
max_words = 10000
maxlen = 500
training_samples = 500
validation_samples = 10000

(input_train, y_train), (input_test, y_test)= imdb.load_data(num_words=max_words)

In [4]:
print("Training Dataset-->>")
print(type(input_train)), print(type(y_train))
print(input_train.shape), print(y_train.shape)
print("\nTest Dataset-->>")
print(type(input_test)), print(type(y_test))
print(input_test.shape), print(y_test.shape)

Training Dataset-->>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(25000,)
(25000,)

Test Dataset-->>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(25000,)
(25000,)


(None, None)

In [5]:
## Already tokenized, so just need to pad_sequences
print(input_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [6]:
from tensorflow.keras.preprocessing import sequence

x_train = sequence.pad_sequences(input_train, maxlen=maxlen)
print(x_train.shape)


x_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print(x_test.shape)

(25000, 500)
(25000, 500)


In [7]:
### In-general we need to make the dtype float and in range (0-1) before feeding to NN beacuse--
# 1. Since here each idx represents a word 
# 2. Secondly we are first passing to Embedding layer which deals with this and gives float values in b/w 0-1. 

print(x_train.shape)
print(x_test.shape)
x_train.dtype, x_train[0][252:312]

(25000, 500)
(25000, 500)


(dtype('int32'),
 array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    1,   14,   22,
          16,   43,  530,  973, 1622, 1385,   65,  458, 4468,   66, 3941,
           4,  173,   36,  256,    5,   25,  100,   43,  838,  112,   50,
         670,    2,    9,   35,  480]))

In [8]:
from tensorflow.keras import Input
from tensorflow.keras import layers, Model

embedding_dim = 100

def simpleRNN(input_shape, num_clases):
    inputs = Input(shape=input_shape)
    
    x = layers.Embedding(max_words, embedding_dim)(inputs)
    x = layers.SimpleRNN(32, return_sequences=True)(x)
    x = layers.SimpleRNN(32)(x)
    
    
    
    if num_clases ==1:
        units = 1
        activation = "sigmoid"
        
    else:
        units=num_clases
        activation = "softmax"
        
    outputs = layers.Dense(units,  activation=activation)(x)
    
    return Model(inputs, outputs)

model = simpleRNN((maxlen,), num_clases=1)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 500, 100)          1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 500, 32)           4256      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 1,006,369
Trainable params: 1,006,369
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(
    optimizer='rmsprop',
    loss="binary_crossentropy",
    metrics=["acc"],
)

# my_callbacks = [
#     tf.keras.callbacks.EarlyStopping(patience=3),
#     tf.keras.callbacks.ModelCheckpoint("../models/simpleRNN.{epoch:02d}_{val_acc:.2f}.h5")
# ]

history = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=10,
    verbose=1,
    validation_split=0.2,

)

Epoch 1/10
157/157 [==============================] - 68s 433ms/step - loss: 0.5160 - acc: 0.7312 - val_loss: 0.4277 - val_acc: 0.8110
Epoch 2/10
157/157 [==============================] - 64s 405ms/step - loss: 0.2989 - acc: 0.8799 - val_loss: 0.4359 - val_acc: 0.8040
Epoch 3/10
157/157 [==============================] - 65s 413ms/step - loss: 0.1759 - acc: 0.9363 - val_loss: 0.4218 - val_acc: 0.8294
Epoch 4/10
157/157 [==============================] - 68s 433ms/step - loss: 0.0758 - acc: 0.9745 - val_loss: 0.6705 - val_acc: 0.7772
Epoch 5/10
157/157 [==============================] - 69s 442ms/step - loss: 0.0321 - acc: 0.9901 - val_loss: 0.7214 - val_acc: 0.8440
Epoch 6/10
157/157 [==============================] - 69s 441ms/step - loss: 0.0159 - acc: 0.9951 - val_loss: 0.9085 - val_acc: 0.7656
Epoch 7/10
157/157 [==============================] - 69s 437ms/step - loss: 0.0097 - acc: 0.9970 - val_loss: 0.9029 - val_acc: 0.7952
Epoch 8/10
157/157 [==============================] - 7